In [1]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from keras_core import utils
from aepy.data.datasets import load_MNIST
from aepy.data.utils import evaluate, display_diff
from aepy.models.base.default_architectures import (VanillaEncoder,VanillaDecoder)
from aepy.models.cae.cae_model import CAE
from aepy.pipelines.training import TrainingPipeline

Using TensorFlow backend


2023-11-29 14:18:37.791066: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

#x_train = np.expand_dims(x_train, -1).astype("float32") / 255
#x_test = np.expand_dims(x_test, -1).astype("float32") / 255

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-11-29 14:18:41 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-11-29 14:18:41 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-11-29 14:18:41 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-11-29 14:18:41 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [3]:
train_data = dict(data=x_train.astype(float), labels=y_train)
test_data = dict(data=x_test.astype(float), labels=y_test)

In [4]:
# Model creation
model = CAE(input_dim=(x_train.shape[0], x_train.shape[1]), latent_dim=2,
            encoder=VanillaEncoder, decoder=VanillaDecoder, layers_conf=[32, 64])

(60000, 28)


In [5]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=1)

trained_model = pipe(train_data=train_data)

2023-11-29 14:18:42 [INFO]: +++ training_pipeline +++
2023-11-29 14:18:42 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-11-29_14-18-42
2023-11-29 14:19:08.807656: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fbed000dab0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-11-29 14:19:08.807708: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-11-29 14:19:08.920907: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1701263950.601400   43767 device_compiler.h:187] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-11-29 14:19:10.604616: E external/local_xla/xla/stream_executor/stream_executor_internal.h:181] SetPriority unimplemented for this stream.
2023-11-29 14:19:10.604716: W external/loc

KeyboardInterrupt: 

In [ ]:
y_hat = trained_model.predict(test_data)

In [ ]:
print(x_test.shape)
print(y_hat['recon'].shape)
display_diff(x_test, y_hat['recon'])